In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
from scipy import stats
from statistics import mean, stdev
from math import sqrt
from matplotlib import pyplot

In [ ]:
# load RNAfold output

rnafold_protein = pd.read_csv('/mnt/sdb1/Project_MIM/Projects_Combined/combined_fasta/rnafold/protein_coding_rnafold_out_GC.csv')
rnafold_nc = pd.read_csv('/mnt/sdb1/Project_MIM/Projects_Combined/combined_fasta/rnafold/noncoding_rnafold_out_GC.csv')

Overview:
===

In [ ]:
rnafold_protein.head(5)

In [ ]:
rnafold_protein.shape

In [ ]:
rnafold_nc.head(5)

In [ ]:
rnafold_nc.shape

General stats:
===

In [ ]:
rnafold_protein.describe()

In [ ]:
rnafold_nc.describe()

In [ ]:
# mead and median are ~ simillar (distribution should be symmetrical)
rnafold_nc['MFE_corrected'].median()

General plots:
===

MFE corrected:
---

In [ ]:
# MFE corrected:

rnafold_protein['MFE_corrected'].plot.density(xlim = (-0.5, 0.0), label = 'Coding', legend = True)
rnafold_nc['MFE_corrected'].plot.density(xlim = (-0.5, 0.0), label = 'Non-coding', legend = True)

In [ ]:
pyplot.figure(dpi=300)
sns.set_context("paper", font_scale=2)   
sns.distplot(rnafold_protein['MFE_corrected'], label = 'Coding', hist=False, color="#018571", kde_kws={"shade": True}) 
sns.distplot(rnafold_nc['MFE_corrected'], label = 'Non-coding', hist=False, color="#a6611a", kde_kws={"shade": True})
plt.legend()
plt.xlim(-0.5, 0.0)

sequence length:
---


In [ ]:
# distribution of sequence length

rnafold_protein['seq_len'].plot.density(xlim = (0.0, 7500.0), label = 'Coding', legend = True) #weights = (np.ones_like(rnafold_protein.index) / len(rnafold_protein.index))*100)
rnafold_nc['seq_len'].plot.density(xlim = (0.0, 7500.0), label = 'Non-coding', legend = True) #weights = (np.ones_like(rnafold_nc.index) / len(rnafold_nc.index))*100)


In [ ]:
pyplot.figure(dpi=300)
sns.set_context("paper", font_scale=2)
sns.distplot(rnafold_protein['seq_len'], label = 'Coding', hist=False, color="green", kde_kws={"shade": True})
sns.distplot(rnafold_nc['seq_len'], label = 'Non-coding', hist=False, color="brown", kde_kws={"shade": True})
plt.xlim(0, 7500)
plt.legend()

GC Content:
---

In [ ]:
# GC content analysis:


rnafold_protein['Percent_GC_countent'].plot.density(label = 'Coding', legend = True)
rnafold_nc['Percent_GC_countent'].plot.density(label = 'Non-coding', legend = True)


In [ ]:
pyplot.figure(dpi=300)
sns.set_context("paper", font_scale=2)
sns.distplot(rnafold_protein['Percent_GC_countent'], label = 'Coding', hist=False, color="green", kde_kws={"shade": True})
sns.distplot(rnafold_nc['Percent_GC_countent'], label = 'Non-coding', hist=False, color="brown", kde_kws={"shade": True})
plt.legend()

TPM:
---

In [ ]:
TPM_IVF_1 = pd.read_csv('/mnt/sdb1/Projects_Combined/RSEM_out_TPM/IVF_1_RSEM_filt_TPM', sep='\t')
TPM_IVF_2 = pd.read_csv('/mnt/sdb1/Projects_Combined/RSEM_out_TPM/IVF_2_RSEM_filt_TPM', sep='\t')

In [ ]:
TPM_IVF_1.set_index('transcript_id', drop=True, inplace=True)
TPM_IVF_2.set_index('transcript_id', drop=True, inplace=True)

In [ ]:
TPM_IVF_1_10 = TPM_IVF_1[TPM_IVF_1['TPM'] > 10]

In [ ]:
TPM_IVF_2_10 = TPM_IVF_2[TPM_IVF_2['TPM'] > 10]

In [ ]:
merged_IVF = pd.merge(TPM_IVF_1, TPM_IVF_2, how='inner', left_index=True, right_index=True)

In [ ]:
merged_IVF_10 = merged_IVF[merged_IVF['TPM_x'] > 0.5]
merged_IVF_10 = merged_IVF[merged_IVF['TPM_y'] > 0.5]

In [ ]:
merged_IVF_10['log_TPM_x'] = np.log2(merged_IVF_10['TPM_x'].values + 1)

In [ ]:
merged_IVF_10['log_TPM_y'] = np.log2(merged_IVF_10['TPM_y'].values + 1)

In [ ]:
merged_IVF_10

In [ ]:
#IVF_1 vs IVF_2 scatterplot:

sns.regplot(x='log_TPM_x', y='log_TPM_y', data=merged_IVF_10)


In [ ]:
np.corrcoef(x=merged_IVF_10['log_TPM_x'], y=merged_IVF_10['log_TPM_y'])

In [ ]:
merged_IVF_10.corr()

In [ ]:
# filtering out NC and proteins. Dataset above ^^ contains ALL mapped reads


IVF_1_TPM_project_plus_wozny = pd.read_csv('/mnt/sdb1/Projects_Combined/RSEM_out_TPM/IVF_1_project_plus_wozny_TPM', sep = '\t')
IVF_1_TPM_project_plus_wozny.set_index('transcript_id', inplace=True)
IVF_1_TPM_project_plus_wozny.head()

In [ ]:
rnafold_nc.rename(columns={'#ID' : 'transcript_id'}, inplace=True)
rnafold_nc.set_index('transcript_id', inplace=True)
rnafold_nc.head()

In [ ]:
rnafold_nc.shape

In [ ]:
IVF_1_NC = pd.merge(IVF_1_TPM_project_plus_wozny, rnafold_nc, right_index=True, left_index=True)

In [ ]:
IVF_1_NC.shape

In [ ]:
IVF_1_NC.columns

In [ ]:
IVF_1_NC.drop(['minimum_free_energy', 'free_energy_of_ensemble', 'centroid_structure', 'seq_len', 'MFE_corrected', 'GC_count', 'GC_content', 'Percent_GC_countent'], axis=1, inplace=True)

In [ ]:
IVF_1_NC.head()

In [ ]:
IVF_2_TPM_project_plus_wozny = pd.read_csv('/mnt/sdb1/Projects_Combined/RSEM_out_TPM/IVF_2_project_plus_wozny_TPM', sep='\t')
IVF_2_NC = pd.DataFrame()

In [ ]:
IVF_2_TPM_project_plus_wozny.set_index('transcript_id', inplace=True)
IVF_2_NC = pd.merge(IVF_2_TPM_project_plus_wozny, rnafold_nc, right_index=True, left_index=True)
IVF_2_NC.drop(['minimum_free_energy', 'free_energy_of_ensemble', 'centroid_structure', 'seq_len', 'MFE_corrected', 'GC_count', 'GC_content', 'Percent_GC_countent'], axis=1, inplace=True)
IVF_2_NC.shape

In [ ]:
IVF_NC_merged = pd.merge(IVF_1_NC, IVF_2_NC, right_index=True, left_index=True, how='outer')

In [ ]:
IVF_NC_merged['log_TPM_x'] = np.log10(IVF_NC_merged['TPM_x'].values + 1)
IVF_NC_merged['log_TPM_y'] = np.log10(IVF_NC_merged['TPM_y'].values + 1)
sns.regplot(x='log_TPM_x', y='log_TPM_y', data=IVF_NC_merged)

In [ ]:
IVF_NC_merged.corr()

In [ ]:
rnafold_protein.rename(columns={'#ID' : 'transcript_id'}, inplace=True)
rnafold_protein.set_index('transcript_id', inplace=True)


In [ ]:
IVF_1_PROT = pd.merge(IVF_1_TPM_project_plus_wozny, rnafold_protein, right_index=True, left_index=True)
IVF_1_PROT.drop(['minimum_free_energy', 'free_energy_of_ensemble', 'centroid_structure', 'seq_len', 'MFE_corrected', 'GC_count', 'GC_content', 'Percent_GC_countent'], axis=1, inplace=True)
IVF_1_PROT.shape

In [ ]:
IVF_2_PROT = pd.merge(IVF_2_TPM_project_plus_wozny, rnafold_protein, right_index=True, left_index=True)

In [ ]:
IVF_2_PROT = pd.merge(IVF_2_TPM_project_plus_wozny, rnafold_protein, right_index=True, left_index=True)
IVF_2_PROT.drop(['minimum_free_energy', 'free_energy_of_ensemble', 'centroid_structure', 'seq_len', 'MFE_corrected', 'GC_count', 'GC_content', 'Percent_GC_countent'], axis=1, inplace=True)
IVF_2_PROT.shape

In [ ]:
IVF_PROT_merged = pd.merge(IVF_1_PROT, IVF_2_PROT, right_index=True, left_index=True, how='outer')

In [ ]:
IVF_PROT_merged.head()

### Porównanie PROT w próbkach IVF1 i IVF2

In [ ]:
IVF_PROT_merged['log10 TPM IVF1'] = np.log10(IVF_PROT_merged['TPM_x'].values + 1)
IVF_PROT_merged['log10 TPM IVF2'] = np.log10(IVF_PROT_merged['TPM_y'].values + 1)
sns.regplot(x='log10 TPM IVF1', y='log10 TPM IVF2', data=IVF_PROT_merged)

In [ ]:
IVF_PROT_merged['log10 TPM IVF1'] = np.log10(IVF_PROT_merged['TPM_x'].values + 1)
IVF_PROT_merged['log10 TPM IVF2'] = np.log10(IVF_PROT_merged['TPM_y'].values + 1)
sns.jointplot(x='log10 TPM IVF1', y='log10 TPM IVF2', data=IVF_PROT_merged, kind='hex')

In [ ]:
sns.kdeplot(np.log10(IVF_PROT_merged['TPM_x'].values + 1), label='log10 TPM IVF1')
sns.kdeplot(np.log10(IVF_PROT_merged['TPM_y'].values + 1), label='log10 TPM IVF2')

###  </> Porównanie PROT w próbkach IVF1 i IVF2

In [ ]:
IVF_PROT_merged.corr()

In [ ]:
IVF_PROT_merged['log_TPM_x'].mean()

In [ ]:
IVF_PROT_merged['log_TPM_y'].mean()

In [ ]:
IVF_NC_merged['log_TPM_x'].mean()

In [ ]:
IVF_NC_merged['log_TPM_y'].mean()

### Porównanie NC i PROT w próbkach MIM1 MIM2

In [ ]:
rnafold_nc = pd.read_csv('/mnt/sdb1/Projects_Combined/combined_fasta/rnafold/noncoding_rnafold_out_GC.csv')
rnafold_nc.rename(columns={'#ID' : 'transcript_id'}, inplace=True)
rnafold_nc.set_index('transcript_id', inplace=True)

In [ ]:

MIM_1_TPM_project_plus_wozny = pd.read_csv('/mnt/sdb1/Projects_Combined/RSEM_out_TPM/MIM_1_project_plus_wozny_TPM', sep = '\t')
MIM_1_TPM_project_plus_wozny.set_index('transcript_id', inplace=True)
MIM_1_NC = pd.merge(MIM_1_TPM_project_plus_wozny, rnafold_nc, right_index=True, left_index=True)
MIM_1_NC.drop(['minimum_free_energy', 'free_energy_of_ensemble', 'centroid_structure', 'seq_len', 'MFE_corrected', 'GC_count', 'GC_content', 'Percent_GC_countent'], axis=1, inplace=True)

MIM_2_TPM_project_plus_wozny = pd.read_csv('/mnt/sdb1/Projects_Combined/RSEM_out_TPM/MIM_2_project_plus_wozny_TPM', sep = '\t')
MIM_2_TPM_project_plus_wozny.set_index('transcript_id', inplace=True)
MIM_2_NC = pd.merge(MIM_2_TPM_project_plus_wozny, rnafold_nc, right_index=True, left_index=True)
MIM_2_NC.drop(['minimum_free_energy', 'free_energy_of_ensemble', 'centroid_structure', 'seq_len', 'MFE_corrected', 'GC_count', 'GC_content', 'Percent_GC_countent'], axis=1, inplace=True)

MIM_NC_merged = pd.merge(MIM_1_NC, MIM_2_NC, right_index=True, left_index=True, how='outer')

MIM_NC_merged['log_TPM_x'] = np.log2(MIM_NC_merged['TPM_x'].values + 1)
MIM_NC_merged['log_TPM_y'] = np.log2(MIM_NC_merged['TPM_y'].values + 1)
sns.relplot(x='log_TPM_x', y='log_TPM_y', data=MIM_NC_merged)


In [ ]:
MIM_NC_merged.corr()

In [ ]:
IVF_PROT_merged.dropna(inplace=True)
IVF_NC_merged.dropna(inplace=True)
MIM_NC_merged.dropna(inplace=True)

In [ ]:
sns.distplot(IVF_PROT_merged['log_TPM_x'], label = 'X', hist=False)
sns.distplot(IVF_NC_merged['log_TPM_x'], label = 'NC', hist=False)
sns.distplot(IVF_PROT_merged['log_TPM_y'], label = 'Y', hist=False)
sns.distplot(IVF_NC_merged['log_TPM_y'], label = 'NC', hist=False)

#sns.distplot(MIM_PROT_merged['log_TPM_x'], label = 'X', hist=False)
#sns.distplot(MIM_NC_merged['log_TPM_x'], label = 'NC', hist=False)
#sns.distplot(MIM_PROT_merged['log_TPM_y'], label = 'Y', hist=False)
#sns.distplot(MIM_NC_merged['log_TPM_y'], label = 'NC', hist=False)


Statistics:
===

MFE corrected:
---

In [ ]:
# according to shapiro the data is not normally distributed, however I'll proceed with Welsch

# This is Welsch test:

stats.ttest_ind(rnafold_protein['MFE_corrected'], rnafold_nc['MFE_corrected'], equal_var = False)

In [ ]:
# function to calculate Cohen's d for independent samples

d1 = rnafold_protein['MFE_corrected'].values
d2 = rnafold_nc_4000['MFE_corrected'].values

def cohend(d1, d2):
    # calculate the size of samples
    n1, n2 = len(d1), len(d2)
    # calculate the variance of the samples
    s1, s2 = np.var(d1, ddof=1), np.var(d2, ddof=1)
    # calculate the pooled standard deviation
    s = sqrt(((n1 - 1) * s1 + (n2 - 1) * s2) / (n1 + n2 - 2))
    # calculate the means of the samples
    u1, u2 = np.mean(d1), np.mean(d2)
    # calculate the effect size

    return (u1 - u2) / s

cohend(d1, d2)

In [ ]:
# Kolmogorov-Smirnov to check if two distributions were the same
# This performs a test of the distribution G(x) of an observed random variable 
# against a given distribution F(x). 
# Under the null hypothesis the two distributions are identical, G(x)=F(x)

from scipy.stats import ks_2samp # kstest for 2 samples

stats.ks_2samp(rnafold_protein['MFE_corrected'], rnafold_nc['MFE_corrected'])

### MFE content confidence interval

In [ ]:
diff_mean = rnafold_protein['GC_content'].mean() - rnafold_nc['GC_content'].mean()
diff_mean

In [ ]:
diff_mean = rnafold_protein['MFE_corrected'].mean() - rnafold_nc['MFE_corrected'].mean()
diff_mean

# df = deegrees of freedom

df = (rnafold_protein['MFE_corrected'].count() + rnafold_nc['MFE_corrected'].count()) - 2
df

t_val = t.ppf([0.975], df)
t_val

N1 = rnafold_protein['MFE_corrected'].count()
N2 = rnafold_nc['MFE_corrected'].count()

std1 = rnafold_protein['MFE_corrected'].std()
std2 = rnafold_nc['MFE_corrected'].count()

std_N1N2 = sqrt(((N1 - 1)*(std1)**2 + (N2 - 1)*(std2)**2) / df) 

MoE = t.ppf(0.975, df) * std_N1N2 * sqrt(1/N1 + 1/N2)
MoE

print ('\nThe difference between groups is {:3.1f} [{:3.1f} to {:3.1f}] (mean [95% CI])'.format(diff_mean, diff_mean - MoE, diff_mean + MoE))

GC Content:
---

In [ ]:
stats.ttest_ind(rnafold_protein['GC_content'], rnafold_nc['GC_content'], equal_var = False)

In [ ]:
stats.ks_2samp(rnafold_protein['GC_content'], rnafold_nc['GC_content'])

### GC content confidence interval



In [ ]:
diff_mean = rnafold_protein['GC_content'].mean() - rnafold_nc['GC_content'].mean()
diff_mean

In [ ]:
# df = deegrees of freedom

df = (rnafold_protein['GC_content'].count() + rnafold_nc['GC_content'].count()) - 2
df

In [ ]:
from scipy.stats import t
t_val = t.ppf([0.975], df)
t_val

In [ ]:
N1 = rnafold_protein['GC_content'].count()
N2 = rnafold_nc['GC_content'].count()

std1 = rnafold_protein['GC_content'].std()
std2 = rnafold_nc['GC_content'].count()

std_N1N2 = sqrt(((N1 - 1)*(std1)**2 + (N2 - 1)*(std2)**2) / df) 

In [ ]:
MoE = t.ppf(0.975, df) * std_N1N2 * sqrt(1/N1 + 1/N2)
MoE

In [ ]:
print ('\nThe difference between groups is {:3.1f} [{:3.1f} to {:3.1f}] (mean [95% CI])'.format(diff_mean, diff_mean - MoE, diff_mean + MoE))

Length:
---

In [ ]:
stats.ttest_ind(rnafold_protein['seq_len'], rnafold_nc['seq_len'], equal_var = False)

In [ ]:
stats.ks_2samp(rnafold_protein['seq_len'], rnafold_nc['seq_len'])

# Statistics for MFE_corrected



In [ ]:
stats.shapiro(rnafold_protein['MFE_corrected'])

In [ ]:
stats.shapiro(rnafold_nc['MFE_corrected'])

In [ ]:
rnafold_nc_4000 = rnafold_nc.head(4000)

In [ ]:
rnafold_nc_4000.describe()

In [ ]:
stats.shapiro(rnafold_nc_4000['MFE_corrected'].sample(50))

In [ ]:
rnafold_nc_4000.plot.hist(label = 'Coding', legend = True, bins=40)

two groups were sampled from different distributions at p < 0.0001

In [ ]:
# kstest, porównanie badanej z rozkladem normalnym, w celu sprawdzenia normalności

stats.kstest(rnafold_protein['MFE_corrected'], 'norm')


In [ ]:
stats.kstest(rnafold_nc['MFE_corrected'], 'norm')

In [ ]:
from statsmodels.stats.diagnostic import lilliefors

lilliefors(rnafold_protein['MFE_corrected'])

In [ ]:
lilliefors(rnafold_nc['MFE_corrected'])

In [ ]:
lilliefors[rnafold_protein['MFE_corrected'], rnafold_nc_4000['MFE_corrected']]

In [ ]:
stats.pearsonr(rnafold_protein['MFE_corrected'], rnafold_nc_4000['MFE_corrected'])

Wszystkie znaki wskazują, że 

In [ ]:
# alternative test for non-normal distributed samples

stats.mannwhitneyu(rnafold_protein['MFE_corrected'], rnafold_nc['MFE_corrected'])

In [ ]:
rnafold_protein_1000 = rnafold_protein.sample(3000)

In [ ]:
lilliefors(rnafold_protein_1000['MFE_corrected'])

In [ ]:
stats.normaltest(rnafold_protein_1000['MFE_corrected'])

In [ ]:
stats.kstest(rnafold_nc['seq_len'], 'norm')

# check for normal distribution (qqplot + hists)

In [ ]:
import numpy as np 
import pylab 
import scipy.stats as stats

## MFE NC

In [ ]:

stats.probplot(rnafold_nc['MFE_corrected'], dist="norm", plot=pylab)
pylab.show()


In [ ]:
rnafold_nc['MFE_corrected'].plot.hist(label = 'Non-coding', legend = True, bins = 40)

## MFE PROT

In [ ]:
stats.probplot(rnafold_protein['MFE_corrected'], dist="norm", plot=pylab)
pylab.show()

In [ ]:
rnafold_protein['MFE_corrected'].plot.hist(label = 'Coding', legend = True, bins=40)

## GC_cont NC

In [ ]:
stats.probplot(rnafold_nc['GC_content'], dist="norm", plot=pylab)
pylab.show()

In [ ]:
rnafold_nc['GC_content'].plot.hist(label = 'Non-coding', legend = True, bins = 40)

## GC_cont PROT

In [ ]:
stats.probplot(rnafold_protein['GC_content'], dist="norm", plot=pylab)
pylab.show()

In [ ]:
rnafold_protein['MFE_corrected'].plot.hist(label = 'Coding', legend = True, bins = 40)

## SEQ_LEN NC

### if not normally dist. = Mann_whitney?

In [ ]:
stats.probplot(rnafold_nc['seq_len'], dist="norm", plot=pylab)
pylab.show()

In [ ]:
rnafold_nc['seq_len'].plot.hist(label = 'Non-coding', legend = True, bins = 40)

## SEQ_LEN PROT

In [ ]:
stats.probplot(rnafold_protein['seq_len'], dist="norm", plot=pylab)
pylab.show()

In [ ]:
rnafold_nc['seq_len'].plot.hist(label = 'Coding', legend = True, bins = 40)

### lncrnanet

In [ ]:
lncrnanet = pd.read_csv('/home/maciek/Pobrane/lncRNAnet/output/lncrnanet_output.txt', sep='\t')
lncrnanet.columns = ['#ID', 'seq_len', 'prob']

In [ ]:
lncrnanet.head()

In [ ]:
lncrnanet_99 = lncrnanet[lncrnanet['prob'] >= 0.998]

In [ ]:
lncrnanet_99.shape

In [ ]:
lncrnanet_99.sort_values(by=['prob'], ascending = False)

In [ ]:
rnafold_nc.head(5)

In [ ]:
nc_lncnet_merged = pd.merge(rnafold_nc, lncrnanet, on="#ID")

In [ ]:
nc_lncnet_merged.sort_values(by='prob', ascending=False)

In [ ]:
nc_lncnet99_merged = pd.merge(rnafold_nc, lncrnanet_99, on='#ID')

In [ ]:
# wpolne dla nc i lncrnanet99

In [ ]:
nc_lncnet99_merged.shape

In [ ]:
nc_lncnet99_merged

### SIGNIFICANT ncRNA from project_combined

In [ ]:
df_significant = pd.read_csv('/mnt/sdb1/Projects_Combined/combined_fasta/cmscan.csv')

In [ ]:
df_significant = df_significant[df_significant['inc'] == '!']

df_significant.to_csv('/mnt/sdb1/Projects_Combined/combined_fasta/cmscan_significant.csv', index=False)